# Lending protocols and short/long leverage data

Load lending market data for a specific chain (Polygon in this example)
and diplay status of the lending market.

In this specific example we download interest rates and prices
for all assets on Aave v3 on Polygon.

## Set up cient and API keys to download dataset

In [1]:
import datetime

from tradingstrategy.client import Client

# You will be interactively prompted to register an API key if you do not have one yet
client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Download datasets

We use a shortcut method `load_trading_and_lending_data` to grab everything we need.

- This function can download current data
- This function can download data for historical time periods

In [2]:
from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradeexecutor.strategy.execution_context import ExecutionMode, ExecutionContext
from tradeexecutor.strategy.trading_strategy_universe import load_trading_and_lending_data, TradingStrategyUniverse

# Download 30 days historical from today
lookback = datetime.timedelta(days=30)

# Load all trading and lending data on Polygon
# for all lending markets on a relevant time period
dataset = load_trading_and_lending_data(
    client,
    execution_context=ExecutionContext(mode=ExecutionMode.data_research),
    universe_options=UniverseOptions(history_period=lookback),
    # Ask for all Polygon data
    chain_id=ChainId.polygon,
    # We limit ourselves to price feeds on Uniswap v3 on Polygon,
    # as there are multiple small or dead DEXes on Polygon
    # which also have price feeds but not interesting liquidity
    exchange_slug="uniswap-v3",
)

# Construct a trading universe that covers all
# chains, DEXes, trading pairs and lending pools we specified
strategy_universe = TradingStrategyUniverse.create_from_dataset(dataset)
data_universe = strategy_universe.data_universe

print(f"We have {data_universe.pairs.get_count()} trading pairs and {data_universe.lending_reserves.get_count()} lending reserves in our datasets")

We have 11 trading pairs and 20 lending reserves in our datasets
